# Open Weather Map API

## 1. create a dummy 'cities' DataFrame

In [ ]:
import pandas as pd

In [ ]:
# sample cities DataFrame for exercises

cities = pd.DataFrame(data = {'city_id' : [1,2,3,4],'city' : ['Berlin', 'Hamburg', 'London', 'Frankfurt'], 'country_code' : ['DE', 'DE', 'GB', 'DE']})
cities

,city_id,city,country_code
0,1,Berlin,DE
1,2,Hamburg,DE
2,3,London,GB
3,4,Frankfurt,DE


In [ ]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   city_id       4 non-null      int64 
 1   city          4 non-null      object
 2   country_code  4 non-null      object
dtypes: int64(1), object(2)
memory usage: 224.0+ bytes


## 2. API key

Don't forget that for OpenWeatherAPI you need an API key

In [ ]:
# please use your own API key
OWM_key = '0866ffee35494a58e8db17d6727c42e5'

## 3. Different approaches

As we saw in the Spotify example, for some API's there could be a Python wrapper that can help you get the data in easier and simpler way.


## 3.2 Approach

OpenWeatherMap API has a nice documentation and you can find it here: https://openweathermap.org/api

We are going to use 5 day / 3 hour forecast https://openweathermap.org/forecast5

Provided at their docmentation you can find an example of API call with only having info about city name and country_code
https://openweathermap.org/forecast5#name5

`api.openweathermap.org/data/2.5/forecast?q={city name},{country code}&appid={API key}`

- city name and country code have to be in format e.g. ('Berin,DE')
- appid is your unique API key
- you can also add units which can be 'standard', 'metric', 'imperial'
- or lang for languages e.g. 'de', 'it' etc.
- or cnt for number of timestamps in response

You can also find an example of API call with having info about latitude and longitude
https://openweathermap.org/forecast5#geo5

api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}

You can choose your own approach. We will show here the approach with city name and country code

#### 3.2.1 Creating a request

In [ ]:
import requests

In [ ]:
weather_dict = {'city_id': [],
                'forecast_time': [],
                'outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'wind_speed': [],
                'pop': []}
for i in range(len(cities['city_id'])):
  url = f"http://api.openweathermap.org/data/2.5/forecast?q={cities['city'][i]},{cities['country_code'][i]}&appid={OWM_key}&units=metric"
  response = requests.get(url)

# iterating over items in 'list' node and selecting the subnodes gives us the exact info we need
  for hour in response.json()['list']:
    weather_dict['city_id'].append(cities['city_id'][i])
    weather_dict['temperature'].append(hour['main']['temp'])
    weather_dict['wind_speed'].append(hour['wind']['speed'])
    weather_dict['forecast_time'].append(hour['dt_txt'])
    weather_dict['outlook'].append(hour['weather'][0]['description'])
    weather_dict['temperature_feels_like'].append(hour['main']['feels_like'])
    weather_dict['pop'].append(hour['pop'])


#### 3.2.2 Exploring the response (json)

You can copy paste your json response output to one of many online json viewers to see your json in nicer and more practical way
http://jsonviewer.stack.hu/

In [ ]:
#This will only work in jupyter notebook so if you are using colab you can skip it

from IPython.display import JSON
# JSON() helps us preview the json in scalable way
JSON(response.json())
# under the node 'list' are the actual weather informations for each hour
JSON(response.json()['list'])

<IPython.core.display.JSON object>

#### 3.2.4 Creating a new weather DataFrame

In [ ]:
weather= pd.DataFrame(weather_dict)
weather

,city_id,forecast_time,outlook,temperature,temperature_feels_like,wind_speed,pop
0,1,2023-03-20 21:00:00,clear sky,8.74,7.05,2.91,0.00
1,1,2023-03-21 00:00:00,scattered clouds,8.27,6.76,2.52,0.00
2,1,2023-03-21 03:00:00,broken clouds,7.71,6.28,2.29,0.00
3,1,2023-03-21 06:00:00,overcast clouds,7.04,5.82,1.92,0.00
4,1,2023-03-21 09:00:00,overcast clouds,8.07,6.55,2.48,0.00
...,...,...,...,...,...,...,...
155,4,2023-03-25 06:00:00,light rain,8.57,5.19,6.70,0.99
156,4,2023-03-25 09:00:00,light rain,8.55,4.91,7.55,0.49
157,4,2023-03-25 12:00:00,light rain,10.81,9.67,7.25,0.86
158,4,2023-03-25 15:00:00,light rain,11.93,10.75,6.10,0.89


In [ ]:
df = pd.json_normalize(response.json())
df["list"]

0    [{'dt': 1679346000, 'main': {'temp': 9.89, 'fe...
Name: list, dtype: object

In [ ]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_id                 160 non-null    int64  
 1   forecast_time           160 non-null    object 
 2   outlook                 160 non-null    object 
 3   temperature             160 non-null    float64
 4   temperature_feels_like  160 non-null    float64
 5   wind_speed              160 non-null    float64
 6   pop                     160 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 8.9+ KB


In [ ]:
weather["forecast_time"]= pd.to_datetime(weather["forecast_time"])

In [ ]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   city_id                 160 non-null    int64         
 1   forecast_time           160 non-null    datetime64[ns]
 2   outlook                 160 non-null    object        
 3   temperature             160 non-null    float64       
 4   temperature_feels_like  160 non-null    float64       
 5   wind_speed              160 non-null    float64       
 6   pop                     160 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 8.9+ KB
